In [1]:
import networkx as nx
from networkx.exception import NetworkXError

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse
import pickle

from SLIM_model import SLIM
import optuna

# データロード

In [2]:
slim_train = pd.read_csv('./data/user_item_train_slim.csv')
triplet_df = pd.read_csv('./data/triplet.csv')
edges = [[r[0], r[1]] for r in triplet_df.values]

user_list = []
item_list = []
entity_list = []
with open('./data/user_list.txt', 'r') as f:
    for l in f:
        user_list.append(l.replace('\n', ''))
with open('./data/item_list.txt', 'r') as f:
    for l in f:
        item_list.append(l.replace('\n', ''))
with open('./data/entity_list.txt', 'r') as f:
    for l in f:
        entity_list.append(l.replace('\n', ''))

In [3]:
len(user_list)

3819

# SLIMでitem_sim_matを作る

In [5]:
# ハイパラ
# gamma
gamma = 0.001
slim = SLIM(gamma, len(user_list), len(item_list))

In [162]:
#slim.fit(slim_train)

In [17]:
slim.save_sim_mat('./sim_mat.txt')

# PageRank

- sim_mat(item_len * item_len)を使って隣接行列を作る
- 隣接行列((item_len + user_len + brand_len) * (item_len + user_len + brand_len))
- nx.google_matrixを参考に隣接行列を作る  


danglingあんまり効果がわからないので注意

In [23]:
G = nx.DiGraph()
G.add_nodes_from([i for i in range(len(entity_list))])
G.add_edges_from(edges)

In [158]:
def google_matrix(G, item_mat=None, alpha=0.85, beta=0.01, personalization=None,
                  weight='weight', dangling=None):

    nodelist = G.nodes()

    M = nx.to_numpy_matrix(G, nodelist=nodelist, weight=weight)
    N = len(G)
    if N == 0:
        return M

    # Personalization vector
    if personalization is None:
        p = np.repeat(1.0 / N, N)
    else:
        missing = set(nodelist) - set(personalization)
        if missing:
            raise NetworkXError('Personalization vector dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        p = np.array([personalization[n] for n in nodelist], dtype=float)
        p /= p.sum()

    #print(p)
    #print(p.shape)
        
    # Dangling nodes
    if dangling is None:
        dangling_weights = p
    else:
        missing = set(nodelist) - set(dangling)
        if missing:
            raise NetworkXError('Dangling node dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        # Convert the dangling dictionary into an array in nodelist order
        dangling_weights = np.array([dangling[n] for n in nodelist],
                                    dtype=float)
        dangling_weights /= dangling_weights.sum()
    dangling_nodes = np.where(M.sum(axis=1) == 0)[0]

    # Assign dangling_weights to any dangling nodes (nodes with no out links)
    for node in dangling_nodes:
        M[node] = dangling_weights

    
    M /= M.sum(axis=1)  # Normalize rows to sum to 1
    
    if item_mat is not None:
        sim_mat = mk_sim_mat(G, item_mat)

        M = beta * M + (1 - beta) * sim_mat
    
    return alpha * M + (1 - alpha) * p

In [160]:
def mk_sim_mat(G, item_mat):
    M = np.eye(len(G.nodes()))
    #M = np.eye(4)
    item_len = item_mat.shape[0]
    M[0:item_len, 0:item_len] = item_mat
    
    # RecWalk論文の定義
    M = M / np.max(M.sum(axis=1)) + np.diag(1 - M.sum(axis=1) / np.max(M.sum(axis=1)))
    return M

In [174]:
def pagerank_numpy(G, item_mat, alpha=0.85, beta=0.01, personalization=None, weight='weight',
                   dangling=None):

    import numpy as np
    if len(G) == 0:
        return {}
    M = google_matrix(G, item_mat, alpha, beta, personalization=personalization,
                      weight=weight, dangling=dangling)
    #return 0
    # use numpy LAPACK solver
    eigenvalues, eigenvectors = np.linalg.eig(M.T)
    ind = eigenvalues.argsort()
    # eigenvector of largest eigenvalue at ind[-1], normalized
    largest = np.array(eigenvectors[:, ind[-1]]).flatten().real
    norm = float(largest.sum())
    return dict(zip(G, map(float, largest / norm)))


In [177]:
val = [0 for i in range(len(G.nodes()))]
k = [i for i in range(len(G.nodes()))]
val[user_idx[0]] = 1
personal = dict(zip(k, val))

# ハイパラ
# alpha, beta
alpha = 0.9
beta = 0.01
pagerank_numpy(G, slim.sim_mat, alpha, beta, personalization=personal)

0

In [214]:
user_idx = [entity_list.index(u) for u in user_list]

def item_ppr(user, alpha, beta):
    val = np.zeros(len(G.nodes()))
    val[user] = 1
    k = [i for i in range(len(G.nodes()))]
    personal_vec = dict(zip(k, val))
    #print(personal_vec)
    #pr = pagerank_numpy(G, slim.sim_mat, alpha, beta, personalization=personal)
    #return pr
    
    # random 後で消す
    # val = np.random.dirichlet([1 for i in range(len(G.nodes))], 1)[0]
    val = np.random.rand(len(G.nodes()))
    val /= val.sum()
    k = [i for i in range(len(G.nodes))]
    ppr = dict(zip(k, val))
    
    pred = []
    item_idx = [entity_list.index(i) for i in item_list]
    for i in item_idx:
        pred.append(ppr[i])
    
    return pred


def get_ranking_mat(alpha=0.85, beta=0.01):
    ranking_mat = []
    count = 0
    for u in user_idx:
        pred = item_ppr(u, alpha, beta)
        #print(pred)
        sorted_idx = np.argsort(np.array(pred))[::-1]
        ranking_mat.append(sorted_idx)
        
        #count += 1
        #if count > 100:
        #    break
            
    return ranking_mat

In [213]:
# ハイパラ
# alpha, beta
alpha = 0.9
beta = 0.01
%time get_ranking_mat()

CPU times: user 3.32 s, sys: 10.1 ms, total: 3.33 s
Wall time: 3.34 s


[array([  40,  733, 1316, ...,  576, 1397,  286]),
 array([ 889, 1137,  980, ..., 1232, 1247, 1334]),
 array([ 333,  477, 1077, ..., 1102,   76,  264]),
 array([  47, 1181,  855, ..., 1221,  439, 1470]),
 array([ 290, 1496, 1206, ..., 1507, 1187, 1361]),
 array([ 409,  253,  481, ..., 1298,  950, 1035]),
 array([1269, 1115, 1053, ...,  520, 1045,  614]),
 array([ 400, 1505, 1495, ...,  952,  837,  772]),
 array([ 705,  295,  138, ..., 1375,  787,  160]),
 array([1444,   57, 1417, ...,  913,  512,  464]),
 array([1039,  184,  902, ..., 1149,  404,  426]),
 array([ 548, 1335,  961, ...,  586, 1141,  231]),
 array([ 268, 1276,  212, ...,  996,  510, 1428]),
 array([1549,   28,  763, ...,  117, 1318, 1198]),
 array([ 467, 1292, 1254, ..., 1188, 1010,  754]),
 array([1361,  512, 1373, ...,   63,  942,  618]),
 array([196, 187,   5, ..., 655, 518, 564]),
 array([ 625,  143,  839, ..., 1412, 1455,   60]),
 array([ 813,  676, 1344, ...,  835,  717, 1134]),
 array([1115,  347, 1514, ...,  442, 

# Evaluate

In [215]:
user_idx = [entity_list.index(u) for u in user_list]
user_items_test_dict = pickle.load(open('./data/user_items_test_dict.pickle', 'rb'))

def topn_precision(ranking_mat, user_items_dict, n=10):
    not_count = 0
    precision_sum = 0
        
    for i in range(len(ranking_mat)):
        if len(user_items_dict[user_idx[i]]) == 0:
            not_count += 1
            continue
        sorted_idx = ranking_mat[i]
        topn_idx = sorted_idx[:n]  
        hit = len(set(topn_idx) & set(user_items_dict[user_idx[i]]))
        precision = hit / len(user_items_dict[user_idx[i]])
        precision_sum += precision
        
    return precision_sum / (len(user_idx) - not_count)

In [216]:
if __name__ == '__main__':
    ranking_mat = get_ranking_mat()
    score = topn_precision(ranking_mat, user_items_test_dict)
    print(score)

0.0054210110745879095
